1. 교재 3.6.3을 참고하여, 배치 처리로 MNIST 테스트 데이터의 정확도를 구해 보시오. p.104 코드에서 x는 학습데이터인가? 테스트데이터인가?

In [27]:
from mnist import load_mnist
import numpy as np
import pickle

def sigmoid(x):
    return 1 / (1 + np.exp(-x))  
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))
def get_data():
    (x_train, t_train), (x_test, t_test) = \
        load_mnist(flatten=True, normalize=True, one_hot_label=False)
    return x_test, t_test
def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        # 학습된 가중치 매개변수가 담긴 파일
        # 학습 없이 바로 추론을 수행
        network = pickle.load(f)

    return network
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y


x, t = get_data()
network = init_network()
accuracy_cnt = 0
batch_size = 100


for i in range(0,len(x),batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch,axis=1)  # 확률이 가장 높은 원소의 인덱스를 얻는다.
    
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))  # Accuracy:0.9352

Accuracy:0.9352


2. 다음은 배치처리에 대한 질문이다. 간단히 답하시오.

 - p.104 코드에서 배치 사이즈가 100이다. 배치 사이즈를 바꾸면 정확도가 달라지는가?

- 배치 처리를 하는 장점은 무엇인가?

크게 달라지지 않습니다.

병렬처리가 가능해 학습,예측이 빠르고 효율적이다. 이 점은 gpu를 사용할 때 더 극대화 됩니다.

3. p.109에서 기술한, 기계학습의 두 가지 접근법은 무엇인가? 차이는? 간단히 적으시오. 

직접 로직을 구현해 규칙성을 사람이 찾아내는 방법

데이터의 특징의  패턴을 인식해 규칙성을 찾아내는 방법

4. 교재에서 기술한, 대표적인 손실 함수 두 가지는 무엇인가? 신경망의 출력 $y$ 와 정답 레이블 $t$ 가 다음과 같을 때, 파이썬 코드를 이용하여 두 손실 함수의 값을 구하시오.$y=(2,0.1,0.2), t=(1,0,0)$

Cross Entropy와 Sum of squares for error

In [9]:
import numpy as np
y = np.array([2,0.1,0.2])
t = np.array([1,0,0])

#SSE
def SSE(y,t):
    return 0.5*np.sum((y-t)**2)
print("SSE: ",SSE(y,t))
#Cross Entropy
def CrossEntropy(y,t):
    return (-1)*np.sum(t*np.log(y))
print("Cross Entropy: ",CrossEntropy(y,t))


SSE:  0.525
Cross Entropy:  -0.6931471805599453


4. 4.2.3의 미니 배치와 3.6.3의 배치는 어떤 차이가 있는가? 미니 배치의 사이즈를 바꾸면 정확도가 달라질 것이라 예상되는가?

미니 배치는 train data중 일부를 골라 학습하는(손실함수의 합을 구하는) 방법입니다. 3.6.3의 배치는 모든 데이터에 대해 손실함수의 합을 구하는 방법입니다.

미니 배치의 사이즈를 바꾸면 정확도가 바뀔 것이라고 생각합니다. 미니배치로 나온 손실값은 전체 손실값의 근사치이므로 이 근사치는 달라질 수 있기 때문입니다.

5. p.121에 수치 미분 함수가 좋지 않은 두 가지 이유는? 그래서 어떻게 개선했는가?

h에 가급적 작은 값을 대입해야 하는데 이때 소수점8자리 이하가 생략되어 0이 대입되기 때문에 계산 결과에 오차가 생깁니다.

$f'(x)$는 $x$일때 $f(x)$의 접선이지만 p.121에서의 미분값은 근사값이기 때문에 정확한 $f'(x)$값이 아닌 $\lim\limits_{h \to 0} \frac{f(x+h)-f(x)}{(x+h) - x}$값이 됩니다. 이때 h를 무한히 0으로 좁히는게 불가능합니다.

따라서  $\lim\limits_{h \to 0} \frac{f(x+h)-f(x-h)}{(x+h) - (x-h)}$ 인 중앙 차분을 이용해 $f'(x)$의 근사치의 오차를 줄입니다.

6. 다음 함수의 편미분을 구하시오.  $f(x_0,x_1) = x_0 + \frac{1}{x_1}$



$(\frac{ \delta f}{\delta x_0} , \frac{ \delta f}{\delta x_1}) =(1, -\frac{1}{x_1^2})$

7. 위 6번 함수에 대해, 손 계산으로 (1, 2)에서의 기울기(gradient)를 구하시오. 

$(1,-\frac{1}{4})$

8. 위 6번 함수에 대해, gradient_2d.py 파일의 함수를 이용하여, (1, 2)에서의 기울기(gradient)를 구하시오. 

In [17]:
def _numerical_gradient_1d(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        
    return grad

def numerical_gradient_2d(f, X):
    if X.ndim == 1:
        return _numerical_gradient_1d(f, X)
    else:
        grad = np.zeros_like(X)
        
        for idx, x in enumerate(X):
            grad[idx] = _numerical_gradient_1d(f, x)
        
        return grad
    
def f(x):
    return x[0]+1/x[1]
print(numerical_gradient_2d(f,np.array([1.0,2.0])))

[ 1.   -0.25]
